# Be Your Own Teacher [ICCV'19]
![model](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbXI72N%2Fbtq6qiAOIN4%2FoKTTxNSy8XsBe41C5ArkpK%2Fimg.jpg)
### 논문 [링크](https://arxiv.org/pdf/1905.08094)
### Pytorch 구현 [링크](https://github.com/luanyunteng/pytorch-be-your-own-teacher)


- 3 Losses

    - Source 1: CrossEntropyLoss btw label & output [Tensorflow](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy)
![Loss source 1](loss1.png)

    - Source 2: KL Divergence btw output[-1] & output[:-1] [Tensorflow](https://www.tensorflow.org/api_docs/python/tf/keras/losses/KLDivergence)
![Loss source 2](loss2.png)

    - Source 3: MSE btw feature[-1] & feature[:-1] [Tensorflow](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError)
![Loss source 3](loss3.png)

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10, cifar100
from tensorflow.keras.layers import Input, Dense, Conv2D, Concatenate, Flatten, Add, MaxPooling2D, GlobalAveragePooling2D, ReLU, BatchNormalization
import numpy as np

In [2]:
gpu_growth = True

if gpu_growth:
    physical_devices = tf.config.list_physical_devices('GPU')
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        # Invalid device or cannot modify virtual devices once initialized.
        pass

In [3]:
def ResBlock(x, filters, strides=1):
    shortcut = x
    
    if strides != 1:
        shortcut = Conv2D(filters=filters, kernel_size=(3,3), strides=strides, padding='same')(shortcut)
        
    x = Conv2D(filters=filters, kernel_size=(3,3), strides=strides, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = ReLU()(x)
    
    x = Conv2D(filters=filters, kernel_size=(3,3), strides=1, padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    
    x = Add()([x,shortcut])
    x = ReLU()(x)
    return x

In [4]:
def BottleNeckLayers(x, filters, kernel, classes):
    features = x
    
    middle_filters = filters // 4
    
    features = Conv2D(filters=middle_filters, kernel_size=1, strides=1)(features)
    features = BatchNormalization(axis=-1)(features)
    features = ReLU()(features)
    
    features = Conv2D(filters=middle_filters, kernel_size=1, strides=1)(features)
    features = BatchNormalization(axis=-1)(features)
    features = ReLU()(features)
    
    features = Conv2D(filters=filters, kernel_size=1, strides=1)(features)
    features = BatchNormalization(axis=-1)(features)
    features = ReLU()(features)
    
    return features

In [7]:
def ResNet18(shape = (32, 32, 3), classes = 10):

    block_layers = [2, 2, 2, 2]
    bottleneck_kernel = [8, 4, 2]
    filters = 64
    
    outputs = { 'output':[], 'features':[] }
    
    # Step 1 Inputs
    x_input = Input(shape)
    
    # Step 2 Initial conv layers
    x = Conv2D(64, kernel_size=7, strides=2, padding='same')(x_input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    
    # Step 3 Add the ResNetBlocks
    for i in range(4):
        for j in range(block_layers[i]):
            strides = 1
            if (i != 0 and j == 0):
                filters *= 2
                strides = 2
            x = ResBlock(x, filters, strides)
            
            if i != 3:
                features = BottleNeckLayers(x, 512, bottleneck_kernel[i], classes)
                outputs['features'].append(features)
                
                fc = GlobalAveragePooling2D()(features)
                fc = Flatten()(fc)
                fc = Dense(classes)(fc)
                outputs['output'].append(fc)
            
                
    outputs['features'].append(x)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(classes)(x)
    outputs['output'].append(x)
    

    model = tf.keras.Model(inputs = x_input, outputs = outputs, name = "ResNet18")
    return model

In [11]:
model = ResNet18((32,32,3), 10)
input_data = np.random.randn(1,32,32,3)
model(input_data)

{'output': [<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
  array([[ 0.21988852, -0.13566482,  0.0539185 , -0.08210978, -0.03606999,
          -0.09465346,  0.0749048 ,  0.12066314,  0.02640524, -0.03018672]],
        dtype=float32)>,
  <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
  array([[-0.03554749,  0.03533215, -0.036688  ,  0.20490134,  0.16054392,
          -0.02079635, -0.0014463 ,  0.1276877 ,  0.13403746,  0.15577944]],
        dtype=float32)>,
  <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
  array([[ 0.01570432, -0.07970074, -0.02652416, -0.06652023, -0.1384543 ,
          -0.03329986, -0.10606084,  0.05591611,  0.03210318,  0.12883194]],
        dtype=float32)>,
  <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
  array([[ 0.1341339 , -0.01684557,  0.10775314,  0.03455899, -0.35720053,
          -0.0069485 ,  0.07947709, -0.10808019,  0.00888537,  0.04256741]],
        dtype=float32)>,
  <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
  array([[-0.01140283,  0.1

# 과제

## 위의 모델을 Tensorflow로 구현하여, Train accuracy 60%, Test accuracy 20%를 달성하여라.
* 다른 여러 Loss들이 있어서 합쳐야 하므로 [GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape)를 사용해서 모델을 만든다

In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, model):
        super(Distiller, self).__init__()
        self.model = model

    def compile(self,
                optimizer, metrics, 
                output_loss_fn,distillation_loss_fn, feature_loss_fn,
                alpha=0.1, temperature=3, lambd=1):
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        ### 
        # TODO: Initialize
        ###

        

    def train_step(self, data):
        # Unpack data
        x, y = data

        
        with tf.GradientTape() as tape:
            # Forward pass of student
            predictions = self.model(x, training=False)
            
            ###
            # TODO: Get outputs from predictions
            # TODO: Compute Losses & Sum Losses
            ###
            output = 

            
            
            # Compute losses

            ## Loss source 1: CrossEntropyLoss btw label & output
            loss = 
            middle1_loss = 
            middle2_loss = 
            middle3_loss = 
            total_loss_1 = 
            
            ## Loss source 2: KL Divergence btw output[:-1] and output[-1]
            loss1by4 =
            loss2by4 =
            loss3by4 = 
            total_loss_2 =
        
            ## Loss source 3: MSE btw Features
            feature_loss_1 =
            feature_loss_2 =
            feature_loss_3 =
            total_loss_3 = 

        # Compute gradients
        
        trainable_vars = self.model.trainable_variables
        gradients = tape.gradient([total_loss_1, total_loss_2, total_loss_3], trainable_vars)
        
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        
        ###
        # TODO: Update state  
        # This will change the output when training
        ###
        
        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, output)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}

        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        ###
        # TODO: Get output from model and calculate accuracy
        ###
        
        # Compute predictions
        predictions = self.model(x, training=False)
        y_prediction = 
        # Calculate the loss
        loss = self.output_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"loss": loss})
        return results

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

###
# TODO: Preprocess Data
###

In [ ]:
shape = x_train.shape[1:]
classes = 100
model = ResNet18(shape,classes)

alpha = 0.1
temperature = 3
lambd = 1e-6

###
# TODO: SET Hyperparameters
###

distiller = Distiller(model)
distiller.compile(


)


In [ ]:
# Distill teacher to student
history = distiller.fit(x_train, y_train, epochs=30)

In [ ]:
# Evaluate student on test dataset
out = distiller.evaluate(x_test, y_test)